In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from model import GPT, GPTConfig
import jax
import equinox as eqx

In [3]:
key = jax.random.PRNGKey(0)

gpt = GPT.create_instance(GPTConfig(), key)

number of parameters: 401.47M


In [5]:
l = GPT.find_sub_tree(gpt, "bias")

In [11]:
set(l)

TypeError: unhashable type: 'jaxlib.xla_extension.ArrayImpl'

In [1]:
whitelist_weight_modules = (eqx.nn.Linear,)
blacklist_weight_modules = (eqx.nn.LayerNorm, eqx.nn.LayerNorm, eqx.nn.Embedding)
decay = set(GPT.find_sub_tree(gpt, "weight", filter_fn=lambda x: isinstance(x, eqx.nn.Linear)))


print(decay)
print("***********************************************************")

NameError: name 'GPT' is not defined